# 📕 MODULE 2.3 : Manipulation des PDF en Python

**Objectif :** Apprendre à lire, extraire du texte et manipuler des fichiers PDF.

Contrairement aux fichiers `.txt`, un PDF est un fichier binaire complexe. Nous utiliserons la bibliothèque **PyPDF2** pour décoder son contenu.

---

## 🛠️ ÉTAPE 1 : Installation et Importation

Nous commençons par installer la bibliothèque nécessaire et l'importer dans notre projet.

In [ ]:
# Installation de la bibliothèque (si elle n'est pas déjà présente)
!pip install PyPDF2

# Importation
import PyPDF2

## 📥 ÉTAPE 2 : Chargement du fichier PDF

Pour travailler, nous avons besoin d'un fichier PDF. Nous allons utiliser `US_Declaration.pdf` comme dans l'exemple du cours.

**Note technique :** Il faut ouvrir le fichier en mode `'rb'` (**R**ead **B**inary) car un PDF n'est pas du texte brut.

In [ ]:
# Si vous êtes sur Colab, téléchargeons d'abord un fichier exemple similaire
# (Si vous avez le fichier 'US_Declaration.pdf' en local, ignorez cette ligne de wget)
!wget -O US_Declaration.pdf https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf

# Ouverture du fichier en mode lecture binaire ('rb')
f = open('US_Declaration.pdf', 'rb')

# Création de l'objet lecteur (PdfReader)
# Note : Dans les versions récentes, PdfFileReader est remplacé par PdfReader
pdf_reader = PyPDF2.PdfReader(f)

## 📊 ÉTAPE 3 : Analyse des métadonnées

Combien de pages contient ce document ? C'est la première chose à vérifier.

In [ ]:
# Affichage du nombre de pages
# Ancienne version : pdf_reader.numPages
# Nouvelle version : len(pdf_reader.pages)
nb_pages = len(pdf_reader.pages)
print(f"Le document contient {nb_pages} page(s).")

## 📝 ÉTAPE 4 : Extraction du texte

Nous allons maintenant extraire le contenu de la **première page** (Index 0).
PyPDF2 permet de transformer le contenu de la page en chaîne de caractères Python.

In [ ]:
# Sélection de la page 0 (la première)
page_one = pdf_reader.pages[0]

# Extraction du texte
page_one_text = page_one.extract_text()

# Affichage du résultat
print("--- DÉBUT DU TEXTE ---")
print(page_one_text)
print("--- FIN DU TEXTE ---")

### 🔎 Analyse du contenu extrait
Comme vous le voyez, le texte extrait peut être analysé comme n'importe quelle chaîne Python.
Nous pouvons par exemple le découper en mots (tokens) pour voir le début.

In [ ]:
# Découpage basique par espace
mots = page_one_text.split()
print(mots[:20]) # Affiche les 20 premiers mots

---

## ✍️ ÉTAPE 5 : Création d'un nouveau PDF

Nous ne pouvons pas modifier un PDF ouvert. Nous devons :
1. Créer un nouvel objet écrivain (`PdfWriter`).
2. Lui ajouter des pages (copiées depuis le lecteur).
3. Sauvegarder le résultat dans un nouveau fichier (`new_file.pdf`).

In [ ]:
# Création de l'écrivain
pdf_writer = PyPDF2.PdfWriter()

# Ajout de la première page récupérée plus haut
pdf_writer.add_page(page_one)

# Ouverture du fichier de destination en mode ÉCRITURE BINAIRE ('wb')
pdf_output = open("new_file.pdf", "wb")

# Écriture effective
pdf_writer.write(pdf_output)

# Fermeture du fichier (très important !)
pdf_output.close()

print("Fichier 'new_file.pdf' créé avec succès.")

---

## 🚜 ÉTAPE 6 : Traitement de tout le document

Souvent, on veut extraire tout le texte d'un coup. Voici comment faire une boucle pour récupérer le texte de toutes les pages.

In [ ]:
f = open('US_Declaration.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(f)

pdf_text = []

# Boucle sur toutes les pages
for p in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[p]
    pdf_text.append(page.extract_text())

f.close()

# Affichage du nombre de pages extraites
print(f"Texte extrait de {len(pdf_text)} pages.")
# print(pdf_text) # Décommentez pour tout voir

## 🧱 ÉTAPE 7 : Fusion de Fichiers (Merger)

Enfin, voyons comment fusionner deux PDF (`F1.pdf` et `F2.pdf`) en un seul document.

In [ ]:
from PyPDF2 import PdfMerger

# Création de fichiers factices pour l'exemple (si vous ne les avez pas)
# (On réutilise new_file.pdf deux fois pour simuler deux fichiers)
filename1 = 'new_file.pdf'
filename2 = 'new_file.pdf'

# Création du fusionneur
merger = PdfMerger()

# Ajout des fichiers dans l'ordre voulu
merger.append(filename1)
merger.append(filename2)

# Sauvegarde du résultat fusionné
merger.write("merged_file.pdf")
merger.close()

print("Fusion terminée : 'merged_file.pdf'")